In [2]:
!pip install ray

     |████████████████████████████████| 51.5 MB 23 kB/s 
     |████████████████████████████████| 127 kB 61.8 MB/s 
     |████████████████████████████████| 3.1 MB 50.7 MB/s 
     |████████████████████████████████| 10.1 MB 52.2 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 72 kB 638 kB/s 
     |████████████████████████████████| 65 kB 4.5 MB/s 
     |████████████████████████████████| 1.3 MB 50.9 MB/s 
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 142 kB 75.2 MB/s 
     |████████████████████████████████| 294 kB 66.7 MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=5559c9fa22097fbdbabe7cf2b1590bc21934d20187c41e066a760106a9f665ed
  Stored in directory: /root/.cache/pip/wheels/e6/67/af/f1ad15974b8fd95f59a63dbf854483ebe5c7a46a93930798b8
Successfully built gpustat


## Ray 철학



https://towardsdatascience.com/modern-parallel-and-distributed-python-a-quick-tutorial-on-ray-99f8d70369b8

병렬처리 모듈은 아래와 같은 요구조건을 원한다. 
- 1개 이상의 머신에서 같은 코드를 실행
- state하고 교류가능한 actor와 마이크로서비스를 구축가능해야한다. 
- 우아하게 오류를 다룰수 있어야한다.
- 효율적으로 큰 객체와 수치데이터를 다루는 것

In [3]:
import pandas as pd
import numpy as np
import psutil
import ray
import cv2
import time

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [4]:
## CPU 개수
CPU_NUM = psutil.cpu_count()
print(CPU_NUM)

2


In [5]:
ray.init(num_cpus = CPU_NUM)

2021-08-05 04:27:23,954	INFO services.py:1247 -- View the Ray dashboard at http://127.0.0.1:8265


{'metrics_export_port': 63178,
 'node_id': '1e8d8cdf830ef7a8b670d0bee3a9695c164355340b21ea7561f49c32',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-08-05_04-27-18_187642_60/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-08-05_04-27-18_187642_60/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-08-05_04-27-18_187642_60',
 'webui_url': '127.0.0.1:8265'}

In [6]:
@ray.remote
def mul(x):
    return x*10

In [8]:
import numpy as np
from multiprocessing import Pool

In [10]:
arr = np.random.random(10**6)
arr.shape

(1000000,)

In [11]:
def mul(x):
    return x*10

In [14]:
# Serial Python
%%time
result = [mul(x) for x in arr]

CPU times: user 572 ms, sys: 22.8 ms, total: 595 ms
Wall time: 598 ms


In [19]:
import ray
ray.__version__

'1.5.1'

In [20]:
## ray
@ray.remote
def mul(x):
    return x*10

In [21]:
arr = ray.put(arr)
arr

ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000001000000)

In [22]:
ray.get(mul.remote(arr))

array([2.08036408, 7.76550683, 9.60719052, ..., 1.81245285, 9.83083586,
       2.09384496])

In [25]:
ray.shutdown()

In [26]:
type(arr)

ray._raylet.ObjectRef

In [ ]:
import numpy as np
import cv2

num_cpus = 2

filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]
aver_ptime = 0
for _ in range(10):
    start = time.time()
    image = np.zeros((3000, 3000))
    result = []
    for i in range(num_cpus):
        result.append(cv2.filter2D(image, -1, filters[i]))
    ptime = time.time() - start
    aver_ptime += ptime / 10
    print(f'{ptime:.3f}s')
print(f'aver ptime : {aver_ptime:.3f}s')

0.275s
0.223s
0.163s
0.179s
0.162s
0.179s
0.167s
0.184s
0.168s
0.168s
aver ptime : 0.187s


In [2]:
import time

In [3]:
start = time.time()
time.sleep(9)
ptime = time.time() - start
print(ptime)

9.009339570999146


In [ ]:
import numpy as np
import cv2
import ray

num_cpus = 2
ray.init(num_cpus = num_cpus, ignore_reinit_error=True)

@ray.remote
def f(image, kernel):
    return cv2.filter2D(image, -1, kernel)

filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

aver_ptime = 0
for _ in range(10):
    start = time.time()
    image = np.zeros((3000, 3000))
    image_id = ray.put(image)
    result = ray.get([f.remote(image_id, filters[i]) for i in range(num_cpus)])
    ptime = time.time() - start
    aver_ptime += ptime / 10
    print(f'{ptime:.3f}s')
print(f'aver ptime : {aver_ptime:.3f}s')

2021-08-04 04:19:18,258	INFO services.py:1247 -- View the Ray dashboard at http://127.0.0.1:8265


0.857s
0.504s
0.381s
0.352s
0.465s
0.259s
0.314s
0.391s
0.259s
0.254s
aver ptime : 0.404s


### 참고사항
https://zzsza.github.io/mlops/2021/01/03/python-ray/

https://medium.com/naver-shopping-dev/ray-%EB%A1%9C-pytorch-model-inference-%ED%95%98%EA%B8%B0-77ce11304604